In [3]:
import sys
import os
from tqdm import tqdm
from time import time, sleep
import gc
import numpy as np
import h5py
import dataclasses
import pandas as pd
from IPython.display import clear_output
from collections import defaultdict
from copy import deepcopy
from PIL import Image

import cv2
import torch
import torch.nn.functional as F
import kornia as K
import kornia.feature as KF

import torch
from lightglue import match_pair
from lightglue import ALIKED, LightGlue
from lightglue.utils import load_image, rbd
from transformers import AutoImageProcessor, AutoModel

import pycolmap
from os.path import join

from hloc.utils.database import *
from hloc.utils.h5_to_db import *
import metric

import argparse
import shutil
from typing import Optional, List, Dict, Any
import multiprocessing
from pathlib import Path
import pycolmap

from hloc.utils.database import COLMAPDatabase
from hloc.triangulation import (
    import_features, import_matches, estimation_and_geometric_verification,
    OutputCapture, parse_option_args)

from hloc import pairs_from_exhaustive
from hloc import extract_features, match_features, match_dense, reconstruction

from hloc.utils import segment
from hloc.utils.io import read_image
from hloc.match_dense import ImagePairDataset

from networks.lightglue.superpoint import SuperPoint
from networks.lightglue.models.matchers.lightglue import LightGlue
from networks.mit_semseg.models import ModelBuilder, SegmentationModule

%load_ext autoreload
%autoreload 2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [13]:


scene_name = "amy_gardens"
version = "gim_lightglue"

images = Path('data/image-matching-challenge-2025/train') / scene_name

outputs = Path('outputs') / scene_name / version

outputs.mkdir(parents=True, exist_ok=True)
os.environ['GIMRECONSTRUCTION'] = str(outputs)

segment_root = Path('outputs') / scene_name / 'segment'
segment_root.mkdir(parents=True, exist_ok=True)

sfm_dir = outputs / 'sparse'
mvs_path = outputs / 'dense'
database_path = sfm_dir / 'database.db'
image_pairs = outputs / 'pairs-near.txt'

feature_conf = matcher_conf = None

if version == 'gim_dkm':
    feature_conf = None
    matcher_conf = match_dense.confs[version]
elif version == 'gim_lightglue':
    feature_conf = extract_features.confs['gim_superpoint']
    matcher_conf = match_features.confs[version]

# Find image pairs via pair-wise image
exhaustive_pairs = pairs_from_exhaustive.main(image_pairs, image_list=images)

if version == 'gim_dkm':
    feature_path, match_path = match_dense.main(matcher_conf, image_pairs,
                                                images, outputs)
elif version == 'gim_lightglue':
    checkpoints_path = join('weights', 'gim_lightglue_100h.ckpt')

    detector = SuperPoint({
        'max_num_keypoints': 2048,
        'force_num_keypoints': True,
        'detection_threshold': 0.0,
        'nms_radius': 3,
        'trainable': False,
    })
    state_dict = torch.load(checkpoints_path, map_location='cpu')
    if 'state_dict' in state_dict.keys(): state_dict = state_dict['state_dict']
    for k in list(state_dict.keys()):
        if k.startswith('model.'):
            state_dict.pop(k)
        if k.startswith('superpoint.'):
            state_dict[k.replace('superpoint.', '', 1)] = state_dict.pop(k)
    detector.load_state_dict(state_dict)
    detector.to(device)

    model = LightGlue({
        'filter_threshold': 0.1,
        'flash': False,
        'checkpointed': True,
    })
    state_dict = torch.load(checkpoints_path, map_location='cpu')
    if 'state_dict' in state_dict.keys(): state_dict = state_dict['state_dict']
    for k in list(state_dict.keys()):
        if k.startswith('superpoint.'):
            state_dict.pop(k)
        if k.startswith('model.'):
            state_dict[k.replace('model.', '', 1)] = state_dict.pop(k)
    model.load_state_dict(state_dict)

    model.to(device)

    feature_path = extract_features.main(feature_conf, images, outputs,
                                            model=detector)
    match_path = match_features.main(matcher_conf, image_pairs,
                                        feature_conf['output'], outputs,
                                        model=model)

[2025/04/26 17:17:30 hloc INFO] Found 19900 pairs.
[2025/04/26 17:17:31 hloc INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 2048, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-gim-superpoint-n2048-r1920',
 'preprocessing': {'grayscale': True, 'resize_max': 1920}}
[2025/04/26 17:17:31 hloc INFO] Found 200 images in root data/image-matching-challenge-2025/train/amy_gardens.
[2025/04/26 17:17:31 hloc INFO] Skipping the extraction.
[2025/04/26 17:17:31 hloc INFO] Matching local features with configuration:
{'model': {'name': 'lightglue', 'weights': 'gim_lightglue_100h'},
 'output': 'matches-gim-lightglue',
 'preprocessing': {'dfactor': 1, 'grayscale': False, 'resize_max': None}}
100%|██████████| 17277/17277 [13:02<00:00, 22.09it/s]
[2025/04/26 17:30:34 hloc INFO] Finished exporting matches.


In [14]:
    # sparse reconstruction
reconstruction.main(sfm_dir, images, image_pairs, feature_path, match_path)

[2025/04/26 17:31:21 hloc INFO] Creating an empty database...
[2025/04/26 17:31:21 hloc INFO] Importing images into the database...
E20250426 17:31:21.812780 136658811417664 images.cc:56] LICENSE.txt BITMAP_ERROR: Failed to read the image file format.
[2025/04/26 17:31:24 hloc INFO] Importing features into the database...
100%|██████████| 200/200 [00:00<00:00, 2306.41it/s]
[2025/04/26 17:31:24 hloc INFO] Importing matches into the database...
100%|██████████| 19900/19900 [00:10<00:00, 1839.10it/s]
[2025/04/26 17:31:35 hloc INFO] Performing geometric verification of the matches...
I20250426 17:31:35.339128 136648310388288 misc.cc:44] 
Feature matching
I20250426 17:31:35.339472 136648301995584 sift.cc:1432] Creating SIFT CPU feature matcher
I20250426 17:31:35.339513 136647798679104 sift.cc:1432] Creating SIFT CPU feature matcher
I20250426 17:31:35.339537 136648293602880 sift.cc:1432] Creating SIFT CPU feature matcher
I20250426 17:31:35.339598 136647807071808 sift.cc:1432] Creating SIFT C

Reconstruction(num_cameras=1, num_images=184, num_reg_images=184, num_points3D=36590)